# Sending mass customized emails with pictures from Google gmail with SMTP in Python

### Preparation

Go to "Manage your Google Account", Search for "Less secure app access", Turn on "Allow less secure apps: ON" to allow Python to access gmail

In Google Contacts, export contacts as Google CSV by default named "contacts.csv"

### In plain plain text, with picture attached

In [ ]:
import pandas as pd
from email.mime.image import MIMEImage
from pathlib import Path
import smtplib, ssl
from string import Template

In [ ]:
# extract family name, email address, title
contacts = pd.read_csv("contacts.csv")[[
    'Family Name', 
    'E-mail 1 - Value', 
    'Organization 1 - Title']]
print('No. of contacts:', len(contacts))
contacts.head()

In [ ]:
# email content
message = """
Dear {title} {family_name}, 

Attached is the picture.


Regards,

maycd
"""

In [ ]:
%%time
# sender's email address
from_address = "sender@email.com"
password = input("Input password: ")

# connect to smtp
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    # login
    server.login(from_address, password)
    
    # loop to ensure the recipient not see other recipients in the address line
    for i in range(len(contacts)):
        # initiate a mime object
        content = MIMEMultipart()
        
        # subject line, sender, recipient address of an email
        content["subject"] = "Email from maycd"
        content["from"] = from_address
        content["to"] = contacts.iloc[i, 1]
        
        # content customized with information in contacts
        message1 = message.format(
            title=contacts.iloc[i, 2], 
            family_name=contacts.iloc[i, 0])
        content.attach(MIMEText(message1))
        
        # image as an attachment
        content.attach(MIMEImage(Path("picture.jpg").read_bytes(),
                                name="picture.jpg"))
        
        # send the email
        server.send_message(content)
        
        # print current process of completion
        print(i, contacts.iloc[i, 2], contacts.iloc[i, 0], contacts.iloc[i, 1])

### In html string, with picture in message body

Note: when recipient's email is Microsoft Outlook, the following method of placing picture in message body seems not applicable.

In [ ]:
# email content
htmlstr = """
Dear $title $family_name, 
<p></p>
<p>Here is the picture.</p>
<p></p>
<p><img src="cid:image1"></p>
<p></p>
<p>Regards,</p>
<p></p>
<p>maycd</p>
"""
# convert to template for substitution
template = Template(htmlstr)

In [ ]:
%%time
from_address = "sender@email.com"
password = input("Input password: ")

context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(from_address, password)
    for i in range(len(contacts)):
        content = MIMEMultipart('alternative')
        content["subject"] = "Email from maycd"
        content["from"] = from_address
        content["to"] = contacts.iloc[i, 1]
        
        # customize the content
        body = template.substitute({ "title": contacts.iloc[i, 2], 
                                    "family_name": contacts.iloc[i, 0] })
        content.attach(MIMEText(body, "html", "utf-8"))
        
        # image placed in text
        fp = open('picture.jpg', 'rb')
        msgImage = MIMEImage(fp.read())
        fp.close()
        msgImage.add_header('Content-ID', '<image1>')
        content.attach(msgImage)
        
        server.send_message(content)
        print(i, contacts.iloc[i, 2], contacts.iloc[i, 0], contacts.iloc[i, 1])